# Decision Tree Implementation from scratch

In [1]:
#Necessary imports
from sklearn import datasets
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import math

In [2]:
#Loading the dataset
iris_data = datasets.load_iris()
print(iris_data.data.shape)
y = iris_data.target.reshape(-1,1)
print(y.shape)

#Concatenating data and taget values using np.hstack
iris = np.hstack((iris_data.data,y))

#Randomly shuffling numpy array
np.random.shuffle(iris)

#Dividing the data into testing and training
training, test = iris[:120,:], iris[120:,:]

print("No. of training examples:", training.shape)
print("No. of testing examples:", test.shape)

(150, 4)
(150, 1)
No. of training examples: (120, 5)
No. of testing examples: (30, 5)


In [3]:
def class_counts(rows):
    """This function will take some rows of dataset and 
will return a dictionary mapping each label with number of rows for that label"""
    d = {}
    for row in rows:
        if row[-1] not in d:
            d[row[-1]]=0
        d[row[-1]]+=1
    return d          

In [4]:
class Question:
    """This class will help in creating Question objects having two properties:
 (i) column/feature and (ii) value for that feature.
 we can have a method in this class to check whether given example matches the value for the 
 feature in this Question or not.
 It will be useful in partitioning the data based on a particular value for a particular feature/column."""
    def __init__(self,col,val):
        self.col = col
        self.val = val
    
    def match(self,example):
        value = example[self.col]
        return value >= self.val    

In [5]:
def partition(rows,question):
    """This function takes list of rows (examples) and after checking for each example, 
    whether it matches the feature value in question or not, it will return two separate list of rows"""
    true_rows,false_rows = [],[]
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows,false_rows

In [6]:
def entropy(rows):
    """This function will take some rows of the dataset and return the entropy of these set of rows.
 It will be used in calculating the info gain for a split based on a question."""
    d = class_counts(rows)
    ent=0
    for i in d:
        v = d[i]/len(rows)
        ent += (-1*v*math.log(v,10))
    return ent

In [7]:
def info_gain(true_rows,false_rows,current_entropy):
    """This function will take the current entropy of the dataset and two different sets of rows after split
 It will calculate the entropy for these two set of rows using entropy function above
 and will return the info gain for that split
 It will be useful in finding the best split for dataset/ given rows."""
    p = len(true_rows)/(len(true_rows)+len(false_rows))
    gain = current_entropy - p*entropy(true_rows) - (1-p)*entropy(false_rows)
    return gain

In [8]:
def best_split(rows):
    """This function will take a set of examples i.e. rows as input
 and return the max info gain and corresponding question."""
    best_gain=0
    best_question = None
    current_entropy = entropy(rows)
    n_features = len(rows[0])-1
    for col in range(n_features): #Iterate over each column
        values = class_counts(rows) 
        for val in values:  #Iterate over each of the unique values fo the column
            question = Question(col,val)
            true_rows,false_rows = partition(rows,question)
            if len(true_rows)==0 or len(false_rows)==0:
                continue
            gain = info_gain(true_rows,false_rows,current_entropy)
            if gain>best_gain:
                best_gain,best_question = gain,question
    return best_gain,best_question #find and return the max gain and best question

In [9]:
class Leaf:
    """Unlike Decision Node, it just gives label for the example that reaches this leaf node.
 It just holds a dictionary mapping as obtained from class_counts function above for given set of rows."""
    def __init__(self,rows):
        self.predictions = class_counts(rows)
        

In [10]:
class Decision_Node:
    """It will have three properties
 (i) question
 (ii) child node representing true branch
 (iii) child node representing false branch"""
    def __init__(self,question,true_branch,false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [11]:
def build_tree(rows,i):
    """This function will build the tree recursively and return the root node of the tree"""
    gain,question = best_split(rows) #find gain and question using best_split function
    print("\nLevel:",i)
    if gain==0: #check for base case i.e. no further split is possible (gain = 0), then return Leaf
        print(f"Count of {rows[0][-1]}:",len(rows))
        print("Current Entropy: 0.0")
        print("Reached Leaf Node")
        return Leaf(rows)
    
    #otherwise do the partition using partition function above and get two list of rows say true_rows and false_rows 
    true_rows,false_rows = partition(rows,question) 
    
    print("Count of 0:",len(false_rows))
    print("Count of 1:",len(true_rows))
    print("Current Entropy:",entropy(rows))
    print("Spliting on feature",question.col)
    
    #Build true_branch and false_branch by recursively calling on true_rows and false_rows
    true_branch = build_tree(true_rows,i+1)
    false_branch = build_tree(false_rows,i+1)
    
    #return the Decision Node with question, true_branch and false_branch
    return Decision_Node(question,true_branch,false_branch)

In [12]:
my_tree = build_tree(training,0)


Level: 0
Count of 0: 39
Count of 1: 81
Current Entropy: 0.4743415670172454
Spliting on feature 2

Level: 1
Count of 0: 61
Count of 1: 20
Current Entropy: 0.2970128977723281
Spliting on feature 3

Level: 2
Count of 2.0: 20
Current Entropy: 0.0
Reached Leaf Node

Level: 2
Count of 1.0: 61
Current Entropy: 0.0
Reached Leaf Node

Level: 1
Count of 0.0: 39
Current Entropy: 0.0
Reached Leaf Node


In [13]:
#OR Tree
train_data = [[0,0,0],
              [0,1,1],
              [1,0,1],
              [1,1,1]]
tree = build_tree(train_data,0)


Level: 0
Count of 0: 2
Count of 1: 2
Current Entropy: 0.24421905028821553
Spliting on feature 0

Level: 1
Count of 1: 2
Current Entropy: 0.0
Reached Leaf Node

Level: 1
Count of 0: 1
Count of 1: 1
Current Entropy: 0.30102999566398114
Spliting on feature 1

Level: 2
Count of 1: 1
Current Entropy: 0.0
Reached Leaf Node

Level: 2
Count of 0: 1
Current Entropy: 0.0
Reached Leaf Node
